In [50]:
from warnings import filterwarnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import CategoricalDtype

In [129]:
def get_data(data):
    if data.split(".")[1] == "txt":
        file = pd.read_csv(data)
    elif data.split(".")[1] == "json":
        file = pd.read_json(data)
    elif data.split(".")[1] == "html":
        file = pd.read_html(data)
    elif data.split(".")[1] == "xml":
        file = pd.xml(data)
    elif data.split(".")[1] == "excel":
        file = pd.excel(data)
    else:
        file = pd.read_csv(data)
    
    data = file.copy()
    return data

In [130]:
df = get_data("energy_dataset.csv").copy()
df.head(5).style.background_gradient()

,time,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,447,329,0,4844,4821,162,0,0,0,nan,863,1051,1899,0,7096,43,73,49,196,0,6378,17,nan,6436,26118,25385,50.1,65.41
1,2015-01-01 01:00:00+01:00,449,328,0,5196,4755,158,0,0,0,nan,920,1009,1658,0,7096,43,71,50,195,0,5890,16,nan,5856,24934,24382,48.1,64.92
2,2015-01-01 02:00:00+01:00,448,323,0,4857,4581,157,0,0,0,nan,1164,973,1371,0,7099,43,73,50,196,0,5461,8,nan,5454,23515,22734,47.33,64.48
3,2015-01-01 03:00:00+01:00,438,254,0,4314,4131,160,0,0,0,nan,1503,949,779,0,7098,43,75,50,191,0,5238,2,nan,5151,22642,21286,42.27,59.32
4,2015-01-01 04:00:00+01:00,428,187,0,4130,3840,156,0,0,0,nan,1826,953,720,0,7097,43,74,42,189,0,4935,9,nan,4861,21785,20264,38.41,56.04


In [45]:
def information_data(df,information="describe"):
    if information == "info":
        print("************************Veri Seti Bilgilendirme****************************")
        return df.info()
    elif information == "dtypes":
        print("****************************Veri Seti Tipleri*******************************")
        return df.dtypes
    elif information == "columns":
        print("***************************Veri Seti Sütunları******************************")
        return df.columns
    elif information == "shape":
        print("**************************Veri Seti Boyutları*******************************")
        return df.shape
    else:
        print("**************************** Veri Seti Özeti *******************************")
        return df.describe().T

In [28]:
def above_zero(val):
    color = 'green' if val > 0 else 'red'
    return 'color: %s' % color

In [29]:
information_data(df).style.applymap(above_zero)

**************************** Veri Seti Özeti *******************************


,count,mean,std,min,25%,50%,75%,max
generation biomass,35045,383.514,85.3539,0,333,367,433,592
generation fossil brown coal/lignite,35046,448.059,354.569,0,0,509,757,999
generation fossil coal-derived gas,35046,0,0,0,0,0,0,0
generation fossil gas,35046,5622.74,2201.83,0,4126,4969,6429,20034
generation fossil hard coal,35046,4256.07,1961.6,0,2527,4474,5838.75,8359
generation fossil oil,35045,298.32,52.5207,0,263,300,330,449
generation fossil oil shale,35046,0,0,0,0,0,0,0
generation fossil peat,35046,0,0,0,0,0,0,0
generation geothermal,35046,0,0,0,0,0,0,0
generation hydro pumped storage aggregated,0,nan,nan,nan,nan,nan,nan,nan


In [33]:
information_data(df,"dtypes")

****************************Veri Seti Tipleri*******************************


time                                            object
generation biomass                             float64
generation fossil brown coal/lignite           float64
generation fossil coal-derived gas             float64
generation fossil gas                          float64
generation fossil hard coal                    float64
generation fossil oil                          float64
generation fossil oil shale                    float64
generation fossil peat                         float64
generation geothermal                          float64
generation hydro pumped storage aggregated     float64
generation hydro pumped storage consumption    float64
generation hydro run-of-river and poundage     float64
generation hydro water reservoir               float64
generation marine                              float64
generation nuclear                             float64
generation other                               float64
generation other renewable                     float64
generation

In [36]:
information_data(df,"columns")

***************************Veri Seti Sütunları******************************


Index(['time', 'generation biomass', 'generation fossil brown coal/lignite',
       'generation fossil coal-derived gas', 'generation fossil gas',
       'generation fossil hard coal', 'generation fossil oil',
       'generation fossil oil shale', 'generation fossil peat',
       'generation geothermal', 'generation hydro pumped storage aggregated',
       'generation hydro pumped storage consumption',
       'generation hydro run-of-river and poundage',
       'generation hydro water reservoir', 'generation marine',
       'generation nuclear', 'generation other', 'generation other renewable',
       'generation solar', 'generation waste', 'generation wind offshore',
       'generation wind onshore', 'forecast solar day ahead',
       'forecast wind offshore eday ahead', 'forecast wind onshore day ahead',
       'total load forecast', 'total load actual', 'price day ahead',
       'price actual'],
      dtype='object')

In [41]:
information_data(df,"shape")

**************************Veri Seti Boyutları*******************************


(35064, 29)

In [46]:
information_data(df,"info")

************************Veri Seti Bilgilendirme****************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 29 columns):
time                                           35064 non-null object
generation biomass                             35045 non-null float64
generation fossil brown coal/lignite           35046 non-null float64
generation fossil coal-derived gas             35046 non-null float64
generation fossil gas                          35046 non-null float64
generation fossil hard coal                    35046 non-null float64
generation fossil oil                          35045 non-null float64
generation fossil oil shale                    35046 non-null float64
generation fossil peat                         35046 non-null float64
generation geothermal                          35046 non-null float64
generation hydro pumped storage aggregated     0 non-null float64
generation hydro pumped storage consumption    350

In [131]:
df_num = df.select_dtypes(include = ["float64", "int64"])
df['time'] = pd.to_datetime(df['time'],utc=True).dt.normalize()
df['date'] = pd.to_datetime(df['time']).dt.date
df['year'] = pd.to_datetime(df['time']).dt.year
df['day'] = pd.to_datetime(df['date']).apply(lambda x: x.day_name())
df['month'] = pd.to_datetime(df['date']).apply(lambda x: x.month_name())
date = df[["date","year","day","month"]]
df = pd.concat([date, df_num], axis=1).copy()
df.head().style.background_gradient()

,date,year,day,month,generation biomass,generation fossil brown coal/lignite,generation fossil coal-derived gas,generation fossil gas,generation fossil hard coal,generation fossil oil,generation fossil oil shale,generation fossil peat,generation geothermal,generation hydro pumped storage aggregated,generation hydro pumped storage consumption,generation hydro run-of-river and poundage,generation hydro water reservoir,generation marine,generation nuclear,generation other,generation other renewable,generation solar,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2014-12-31,2014,Wednesday,December,447,329,0,4844,4821,162,0,0,0,nan,863,1051,1899,0,7096,43,73,49,196,0,6378,17,nan,6436,26118,25385,50.1,65.41
1,2015-01-01,2015,Thursday,January,449,328,0,5196,4755,158,0,0,0,nan,920,1009,1658,0,7096,43,71,50,195,0,5890,16,nan,5856,24934,24382,48.1,64.92
2,2015-01-01,2015,Thursday,January,448,323,0,4857,4581,157,0,0,0,nan,1164,973,1371,0,7099,43,73,50,196,0,5461,8,nan,5454,23515,22734,47.33,64.48
3,2015-01-01,2015,Thursday,January,438,254,0,4314,4131,160,0,0,0,nan,1503,949,779,0,7098,43,75,50,191,0,5238,2,nan,5151,22642,21286,42.27,59.32
4,2015-01-01,2015,Thursday,January,428,187,0,4130,3840,156,0,0,0,nan,1826,953,720,0,7097,43,74,42,189,0,4935,9,nan,4861,21785,20264,38.41,56.04


In [133]:
def time_prapared():
    month_category = ["January","February","March", "April",
                  "May","June","July","August","September","October","November","December"]
    df.month = df.month.astype(CategoricalDtype(categories = month_category, ordered = True))
    day_category = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
    df.day = df.day.astype(CategoricalDtype(categories = day_category, ordered = True))
    return df

In [134]:
df = time_prapared().copy()
df.month.head().unique()

[December, January]
Categories (2, object): [January < December]

In [121]:
df.day.head().unique()

[Wednesday, Thursday]
Categories (2, object): [Wednesday < Thursday]

In [122]:
df.year.head().unique()

array([2014, 2015], dtype=int64)

In [123]:
df.groupby("day")["price actual"].count()

day
Monday       5015
Tuesday      4992
Wednesday    4993
Thursday     5016
Friday       5016
Saturday     5016
Sunday       5016
Name: price actual, dtype: int64

In [124]:
df.groupby("month")["price actual"].count()

month
January      2976
February     2712
March        2976
April        2880
May          2976
June         2880
July         2976
August       2976
September    2880
October      2976
November     2880
December     2976
Name: price actual, dtype: int64

In [125]:
df.groupby("year")["price actual"].count()

year
2014       1
2015    8760
2016    8784
2017    8760
2018    8759
Name: price actual, dtype: int64

In [126]:
df.groupby("year")["price actual"].mean()

year
2014    65.410000
2015    61.359949
2016    47.438307
2017    59.321965
2018    63.444265
Name: price actual, dtype: float64

In [128]:
df.groupby("month")["price actual"].mean()

month
January      61.634143
February     53.305052
March        47.867644
April        48.288017
May          51.889509
June         58.213566
July         60.392382
August       59.015988
September    60.850993
October      63.483149
November     63.851729
December     65.399200
Name: price actual, dtype: float64

In [127]:
df.groupby("day")["price actual"].mean()

day
Monday       59.463208
Tuesday      60.097957
Wednesday    59.911003
Thursday     60.066699
Friday       59.676298
Saturday     54.639753
Sunday       51.353445
Name: price actual, dtype: float64